In [1]:
### import necessary libraries
import ee
import json
import geemap
import sklearn
import numpy as np
import pandas as pd
import geopandas as gpd
from pprint import pprint
import matplotlib.pyplot as plt

ee.Authenticate()
ee.Initialize(project="ee-franciscofurey")

In [2]:
def refined_lee(img):
    """
    Applies the Refined Lee speckle filter.
    
    Args:
        img: The ee.Image to filter, must be in natural units, not dB.
    
    Returns:
        The filtered ee.Image.
    """
    # Set up 3x3 kernels
    weights3 = ee.List.repeat(ee.List.repeat(1, 3), 3)
    kernel3 = ee.Kernel.fixed(3, 3, weights3, 1, 1, False)

    mean3 = img.reduceNeighborhood(ee.Reducer.mean(), kernel3)
    variance3 = img.reduceNeighborhood(ee.Reducer.variance(), kernel3)

    # Use a sample of the 3x3 windows inside a 7x7 windows to determine gradients and directions
    sample_weights = ee.List([[0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0], 
                              [0, 1, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 1, 0], 
                              [0, 0, 0, 0, 0, 0, 0]])
    sample_kernel = ee.Kernel.fixed(7, 7, sample_weights, 3, 3, False)

    sample_mean = mean3.neighborhoodToBands(sample_kernel)
    sample_var = variance3.neighborhoodToBands(sample_kernel)

    # Determine the 4 gradients for the sampled windows
    gradients = sample_mean.select(1).subtract(sample_mean.select(7)).abs()
    gradients = gradients.addBands(sample_mean.select(6).subtract(sample_mean.select(2)).abs())
    gradients = gradients.addBands(sample_mean.select(3).subtract(sample_mean.select(5)).abs())
    gradients = gradients.addBands(sample_mean.select(0).subtract(sample_mean.select(8)).abs())

    # Find the maximum gradient amongst gradient bands
    max_gradient = gradients.reduce(ee.Reducer.max())

    # Create a mask for band pixels that are the maximum gradient
    gradmask = gradients.eq(max_gradient)

    # Duplicate gradmask bands: each gradient represents 2 directions
    gradmask = gradmask.addBands(gradmask)

    # Determine the 8 directions
    directions = sample_mean.select(1).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(7))).multiply(1)
    directions = directions.addBands(sample_mean.select(6).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(2))).multiply(2))
    directions = directions.addBands(sample_mean.select(3).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(5))).multiply(3))
    directions = directions.addBands(sample_mean.select(0).subtract(sample_mean.select(4)).gt(sample_mean.select(4).subtract(sample_mean.select(8))).multiply(4))
    # The next 4 are the not() of the previous 4
    directions = directions.addBands(directions.select(0).Not().multiply(5))
    directions = directions.addBands(directions.select(1).Not().multiply(6))
    directions = directions.addBands(directions.select(2).Not().multiply(7))
    directions = directions.addBands(directions.select(3).Not().multiply(8))

    # Mask all values that are not 1-8
    directions = directions.updateMask(gradmask)

    # Collapse the stack into a single band image
    directions = directions.reduce(ee.Reducer.sum())

    sample_stats = sample_var.divide(sample_mean.multiply(sample_mean))

    # Calculate localNoiseVariance
    sigmaV = sample_stats.toArray().arraySort().arraySlice(0, 0, 5).arrayReduce(ee.Reducer.mean(), [0])

    # Set up the 7*7 kernels for directional statistics
    rect_weights = ee.List.repeat(ee.List.repeat(0, 7), 3).cat(ee.List.repeat(ee.List.repeat(1, 7), 4))
    diag_weights = ee.List([[1, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 0], [1, 1, 1, 0, 0, 0, 0], 
                            [1, 1, 1, 1, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 0], 
                            [1, 1, 1, 1, 1, 1, 1]])
    rect_kernel = ee.Kernel.fixed(7, 7, rect_weights, 3, 3, False)
    diag_kernel = ee.Kernel.fixed(7, 7, diag_weights, 3, 3, False)

    dir_mean = img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel).updateMask(directions.eq(1))
    dir_var = img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel).updateMask(directions.eq(1))

    # Rotate kernels and add bands for mean and variance
    for i in range(1, 4):
        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), rect_kernel.rotate(i)).updateMask(directions.eq(2 * i + 1)))
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), rect_kernel.rotate(i)).updateMask(directions.eq(2 * i + 1)))
        dir_mean = dir_mean.addBands(img.reduceNeighborhood(ee.Reducer.mean(), diag_kernel.rotate(i)).updateMask(directions.eq(2 * i + 2)))
        dir_var = dir_var.addBands(img.reduceNeighborhood(ee.Reducer.variance(), diag_kernel.rotate(i)).updateMask(directions.eq(2 * i + 2)))

    dir_mean = dir_mean.reduce(ee.Reducer.sum())
    dir_var = dir_var.reduce(ee.Reducer.sum())

    varX = dir_var.subtract(dir_mean.multiply(dir_mean).multiply(sigmaV)).divide(sigmaV.add(1.0))

    b = varX.divide(dir_var)

    result = dir_mean.add(b.multiply(img.subtract(dir_mean)))
    return result.arrayFlatten([['sum']])


In [5]:

# Define dates
before_start = '2018-07-15'
before_end = '2018-08-10'
after_start = '2018-08-10'
after_end = '2018-08-23'

admin2 = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2")
ernakulam = admin2.filter(ee.Filter.eq('ADM2_NAME', 'Ernakulam'))
s1 = ee.ImageCollection("COPERNICUS/S1_GRD")
geometry = ernakulam.geometry()
Map = geemap.Map()
# Load the Ernakulam district geometry
Map.addLayer(geometry, {'color': 'grey'}, 'Ernakulam District')

# Filter the image collection
collection = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))\
    .filter(ee.Filter.eq('resolution_meters', 10))\
    .filter(ee.Filter.bounds(geometry))\
    .select('VH')

# Filter collections for before and after the specified dates
before_collection = collection.filter(ee.Filter.date(before_start, before_end))
after_collection = collection.filter(ee.Filter.date(after_start, after_end))

# Mosaic and clip images
before = before_collection.mosaic().clip(geometry)
after = after_collection.mosaic().clip(geometry)

# Center map and add layers
Map.centerObject(geometry, 10)
Map.addLayer(before, {'min': -25, 'max': 0}, 'Before Floods', False)
Map.addLayer(after, {'min': -25, 'max': 0}, 'After Floods', False)

# Refined Lee Filter (Kernel)

# Speckle Filtering Functions
def to_natural(img):
    return ee.Image(10.0).pow(img.select(0).divide(10.0))

def to_db(img):
    return ee.Image(img).log10().multiply(10.0)

before_filtered = ee.Image(to_db(refined_lee(to_natural(before))))
after_filtered = ee.Image(to_db(refined_lee(to_natural(after))))

Map.addLayer(before_filtered, {'min': -25, 'max': 0}, 'Before Floods Filtered', False)
Map.addLayer(after_filtered, {'min': -25, 'max': 0}, 'After Floods Filtered', False)

# Exercise
# A simple method for filtering speckles is using a focal median filter
# Apply a Focal Median filter on both before and after images
# Use a Circle kernel with a 30 meter radius
# Add the filtered images to the map
# Hint: Use the foal_median() function

# Apply Focal Median filter on before image
before_filtered = before.focal_median(radius=30, units='meters')

# Apply Focal Median filter on after image
after_filtered = after.focal_median(radius=30, units='meters')

# Add filtered images to the map
Map.addLayer(before_filtered, {'min': -25, 'max': 0}, 'Before Floods Filtered', False)
Map.addLayer(after_filtered, {'min': -25, 'max': 0}, 'After Floods Filtered', False)


In [6]:
Map

Map(center=[10.055341788485276, 76.46953395211857], controls=(WidgetControl(options=['position', 'transparent_…